In [4]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


#  Setup stuff: Connectivity

In [5]:

from katana import remote
from katana.remote import import_data


my_client = remote.Client()

print(my_client)


In [6]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [7]:

#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


--


In [ ]:

#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:

#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [8]:

#  CREATE GRAPH

my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph, 4LHA2M3VNGUmEBGVUDm11gv6uvi3bZV8W5P6zsSWf2sD, 0>


In [9]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


<_Graph my_graph, 4LHA2M3VNGUmEBGVUDm11gv6uvi3bZV8W5P6zsSWf2sD, 0>


# Create: Vertices/nodes, edges ..

In [ ]:
# import pandas as pd
# 
# print("--")


In [10]:

#  Vertices/Nodes, Airports ..

df_airports = pd.DataFrame([
       #
   ["MKE", "Milwaukee"               , "Airport"],
   ["ORD", "Chicago O-Hare"          , "Airport"],
   ["SJC", "San Jose"                , "Airport"],
   ["DEN", "Denver"                  , "Airport"],
       #
   ], columns = ["id", "airport_name", "LABEL"])

print(tabulate(df_airports, headers='keys', tablefmt='psql'))


+----+------+----------------+---------+
|    | id   | airport_name   | LABEL   |
|----+------+----------------+---------|
|  0 | MKE  | Milwaukee      | Airport |
|  1 | ORD  | Chicago O-Hare | Airport |
|  2 | SJC  | San Jose       | Airport |
|  3 | DEN  | Denver         | Airport |
+----+------+----------------+---------+


In [11]:
#  Create the edge, flights ..

df_flights = pd.DataFrame([
      #
   ["MKE", "ORD",   66, 1, "FLIES_TO" ],
   ["ORD", "MKE",   66, 1, "FLIES_TO" ],
      #
   ["ORD", "DEN",  886, 1, "FLIES_TO" ],
   ["DEN", "ORD",  886, 1, "FLIES_TO" ],
      #
   ["SJC", "DEN",  948, 1, "FLIES_TO" ],                           #  Notice SJC flies to/from Denver
   ["DEN", "SJC",  948, 1, "FLIES_TO" ],
      #
   ["SJC", "ORD", 1829, 1, "FLIES_TO" ],                           #  Notice SJC flies to ORD, but not ORD to SJC
      #
   ], columns = ["START_ID", "END_ID", "DISTANCE", "NUM_HOPS", "TYPE"])

print(tabulate(df_flights, headers='keys', tablefmt='psql'))


+----+------------+----------+------------+------------+----------+
|    | START_ID   | END_ID   |   DISTANCE |   NUM_HOPS | TYPE     |
|----+------------+----------+------------+------------+----------|
|  0 | MKE        | ORD      |         66 |          1 | FLIES_TO |
|  1 | ORD        | MKE      |         66 |          1 | FLIES_TO |
|  2 | ORD        | DEN      |        886 |          1 | FLIES_TO |
|  3 | DEN        | ORD      |        886 |          1 | FLIES_TO |
|  4 | SJC        | DEN      |        948 |          1 | FLIES_TO |
|  5 | DEN        | SJC      |        948 |          1 | FLIES_TO |
|  6 | SJC        | ORD      |       1829 |          1 | FLIES_TO |
+----+------------+----------+------------+------------+----------+


In [13]:

# Import the 2 previously created Python DataFrames into KatanaGraph ..

with import_data.DataFrameImporter(my_graph) as df_importer:   
    
   df_importer.nodes_dataframe(df_airports,                    #  Aiports set of Nodes
      id_column             = "id",
      id_space              = "Airport",  
      label                 = "Airport",  
      )
   
   df_importer.edges_dataframe(df_flights,                     #  Our Edge, specifying the relationship between Airport --> FLIES_TO --> Airport
      source_id_space       = "Airport", 
      destination_id_space  = "Airport",   
      source_column         = "START_ID",
      destination_column    = "END_ID",
      type                  = "FLIES_TO"
      )

   df_importer.insert()
    
print("--")


          0/? [?op/s]

          0/? [?op/s]

--


#  Diagnostics

In [14]:

#  What's in the graph ..

l_result1 = my_graph.query("""
   MATCH ( n ) 
   RETURN n
   """)
print(tabulate(l_result1, headers='keys', tablefmt='psql'))

l_result2 = my_graph.query("""
   MATCH ( n ) - [ r ] -> ( m )
   RETURN r
   """)
print(tabulate(l_result2, headers='keys', tablefmt='psql'))


          0/? [?op/s]

+----+-----------------+-------------+-----------+------------------+--------+----------+
|    |   n.internal_id | n.labels    | n.LABEL   | n.airport_name   | n.id   | n.type   |
|----+-----------------+-------------+-----------+------------------+--------+----------|
|  0 |               2 | ['Airport'] | Airport   | San Jose         | SJC    | node     |
|  1 |               3 | ['Airport'] | Airport   | Denver           | DEN    | node     |
|  2 |               1 | ['Airport'] | Airport   | Chicago O-Hare   | ORD    | node     |
|  3 |               0 | ['Airport'] | Airport   | Milwaukee        | MKE    | node     |
+----+-----------------+-------------+-----------+------------------+--------+----------+


          0/? [?op/s]

+----+------------+----------------+-----------------+-----------+--------------+--------------+----------+--------------+------------------+----------+
|    |   r.end.id | r.end.labels   |   r.internal_id | r.label   |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   |   r.start.id | r.start.labels   | r.type   |
|----+------------+----------------+-----------------+-----------+--------------+--------------+----------+--------------+------------------+----------|
|  0 |          1 | ['Airport']    |               1 | FLIES_TO  |           66 |            1 | FLIES_TO |            0 | ['Airport']      | edge     |
|  1 |          1 | ['Airport']    |               2 | FLIES_TO  |         1829 |            1 | FLIES_TO |            2 | ['Airport']      | edge     |
|  2 |          3 | ['Airport']    |               6 | FLIES_TO  |          948 |            1 | FLIES_TO |            2 | ['Airport']      | edge     |
|  3 |          1 | ['Airport']    |               3 | FLIES_TO  |          886 | 

#  at_version(), (snapshots) ..

In [17]:

my_graph.get_snapshots()





AttributeError: '_Database' object has no attribute 'graphs'

In [ ]:
# ##################################################################

In [ ]:
#  Point to a prior version of our graph ..
#
#     Future, also later in time; add Client.transaction()

l_version = 5

my_graph2 = my_graph.at_version(l_version)

print("--")


In [ ]:
if 'my_graph2' in globals():
   display(my_graph2.num_nodes())
   display(my_graph2.num_edges())
else: 
   print("--")


In [ ]:

#  Used to test heterogeneous graphs ..

l_query = """
   CREATE ( n: Restaurant { restaurant_code: 'PAP' } )               //  Notice single curly braces
   SET n.restaurant_name = 'Pappadeauxs Seafood Kitchen' 
   """
      #
display(l_result1 = my_graph.query(l_query))

l_query = """
   MATCH
      (n: Airport   ),
      (m: Restaurant)
   WHERE n.id               = 'SJC'
   AND   m.restaurant_code  = 'PAP'
   CREATE (n) -[r: Attraction { DRIVABLE: 'Yes' }]-> (m)
   """
      #
display(l_result2 = my_graph.query(l_query))
